# HW3

In [382]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('talk')
%matplotlib notebook
%config InlineBackend.figure_format = 'retina'

## Problem 1: Linear Regression Class [40pts]

### Part 1: Base Class

In [87]:
class Regression(object):
    
    def __init__(self):
        self.params = {}
    
    def get_params(self):
        # return best fit params (B-hat) from fit()
        return self.params
    
    def set_params(self, **kwargs):
        # TODO
        pass
    
    def fit(self, X, y):
        # fits linear model supplied from subclass to X and y 
        # and stores best fit coeffs (b_hat) in params dict
        """Fit model."""
    
    def predict(self, X):
        # TODO
        pass 
    
    def score(self, X, y):
        # TODO
        pass

### Part 2: OLS Linear Regression

In [753]:
class LinearRegression(Regression):
    
    def __init__(self):
        self.M = 0
        super().__init__() # to maintina access to params dict
    #    self.M = M # for Ridge. M=0 for OLS by default
        
    def fit(self, X, y):
        #self.M = 0 # for Ridge. M=0 for OLS by default
        # create design matrix (padding with ones column)
        if X.ndim == 1: # just duplicate the same observation
            X = np.insert(X, 0, 1)
            X = np.tile(X, (len(X), 1))
        else:
            num_rows = X.shape[0]
            intercept_coeffs = np.ones(num_rows)
            X = np.column_stack((intercept_coeffs, X))
        
        # compute B_hat
        X_inv = np.linalg.pinv(X)
        y = np.array([y]).T # convert to column vector
        B_hat = np.linalg.pinv(X.T.dot(X) + self.M).dot(X.T).dot(y)
        # return params
        B_hat = B_hat.ravel() # convert to 1D array
        self.params['intercept'] = B_hat[0]
        self.params['coeffs'] = B_hat[1:]

### Part 3: Ridge Regression

In [693]:
class RidgeRegression(LinearRegression):
    def __init__(self, alpha):
        super().__init__() # get access to params dict
        self.alpha = alpha
        self.M = self.get_M() # overwrite default OLS M=0
    #Gamma = self.alpha*np.identity(len(X.T.dot(X))) # Ridge
    #M = Gamma.
    #M = 2
    
    def get_M(self):
        # Gamma = alpha * I
        # The +1 accounts for extra column of 1's to give I the
        # right dimensions
        Gamma = self.alpha*np.identity(len(X.T.dot(X)) + 1)
        return Gamma.T.dot(Gamma)

In [377]:
ridge_fit = RidgeRegression(alpha=0)
ridge_fit.fit(X, y)
ridge_fit.get_params()

{'intercept': -0.6194165338507149, 'coeffs': array([0.50422701, 1.52991823])}

### Part 4: Model Scoring

###### Import the data

In [485]:
from sklearn import datasets
dataset = datasets.load_boston()

###### Poke around inside

In [486]:
# just looking at everything stored inside the dict
width = 20
for k, v in dataset.items():
    print(f"{k} {np.shape(v)}:\n")

data (506, 13):

target (506,):

feature_names (13,):

DESCR ():



###### Let's take a look inside the "empty" `DESCR` field 

In [ ]:
print(dataset['DESCR'])

Boston House Prices dataset

Notes
------
Data Set Characteristics:  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive
    
    :Median Value (attribute 14) is usually the target

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
      

######  The dataset description in `dataset['DESR']` tells us that there are 14 columns, but there are only 13 in `dataset.data` according to its shape though. This means that the 14th column is probably stored in the only other key whose value is a 1-D array, `dataset['target']` . This is also noted as "Median Value (attribute 14) is usually the target" in the `DESCR` field. I'll just tack this on to the end of the 2-D array and display it as a `pandas` DataFrame

In [ ]:
import pandas as pd
pd.options.display.max_rows = 10 # reduce how many rows are shown

# I'm not using dot notation to access data and feature names because I am pretending that dataset is a dict and not an sklearn object for this HW.
df = pd.DataFrame(dataset['data'], columns=dataset['feature_names'])
target = 'target'
df[target] = dataset[target] 
df.iloc[0]

CRIM         0.00632
ZN          18.00000
INDUS        2.31000
CHAS         0.00000
NOX          0.53800
             ...    
TAX        296.00000
PTRATIO     15.30000
B          396.90000
LSTAT        4.98000
target      24.00000
Name: 0, Length: 14, dtype: float64

###### Ok, so it looks like each row is the stats for a town (or collection of homes?) and each column are some corresponding stats, with the last one being the median price in that particular  town in the  $1000's$.  

#### Regression

###### Recasting this as a linear regression problem: for any given town $i$, $y_i$ is the observed target median price corresponding to homes in that town and $x_{ij}$ is the $j$th statistic that was also measured for town $i$, corresponding to the $j$ column in row $i$ of the table shown above. For the $p$ regressors (number of columns, not including the target column), 

\begin{align}
    y_i = \sum_{j=0}^{p} x_{ij}\beta_j + \epsilon_i\ , 
    (i = 1,2,\dots, m)\ ,
\end{align}

###### where  $m$ is the number of towns (observations), $\beta_i$ is some scalar coefficient, $\epsilon_i$ is/are some unobserved random variable/s that account for influences on $y_i$ other than the collection of regressors $x_i$ that we happen to know about. Setting $x_{i0} = 0$ allows $(\beta_0)$ to act  as our intercept. In matrix form, all of the $m$ models (number of rows in the table) can compactly be written as
\begin{align}
    \boldsymbol y = \boldsymbol X \boldsymbol\beta + \boldsymbol\epsilon
    \ ,
\end{align}

###### where $\boldsymbol y$ is a column vector with length $m$, $\boldsymbol X$ is an $m\times n$ (design) matrix (where $n=p+1$, since it's just the data table above with a first column of ones and minus the target column), and $\boldsymbol\beta$ and $\boldsymbol\epsilon$ are also column  vectors of length $m$ composed of the $\beta_i$'s and $\epsilon_i$'s mentioned above. 

##### Game plan

###### Alright, I'm going to use the first 405 models ($\approx 80\%$ of the data) as my training set to determine the best fit coefficients $\boldsymbol{\hat\beta}$ found from OLS and Ridge to test on the reminaing 101 models, respectively, and score how well each predictive model does on the remaing 101 models in predicting the target housing price $y_i$.

In [754]:
num_models = 405
training_input = dataset['data'][0:num_models, :]
training_output = dataset['target'][0:num_models]
X = training_input
y = training_output

lin_fit = LinearRegression()
lin_fit.fit(X, y)
lin_fit.get_params()

{'intercept': 30.1834801417563,
 'coeffs': array([-1.94651664e-01,  4.40677436e-02,  5.21447706e-02,  1.88823450e+00,
        -1.49475195e+01,  4.76119492e+00,  2.62339333e-03, -1.30091291e+00,
         4.60230476e-01, -1.55731325e-02, -8.11248033e-01, -2.18154708e-03,
        -5.31513940e-01])}

## Problem 2 [10pts]

### Part 1: Create a module

###### Included in directory as `MathCS207.py`

### Part 2: Import a whole module and use it

In [48]:
import MathCS207

# inputs
inputs = {'a':1., 'b':2.}

# show output from module
print(
    f"inputs: {inputs}\n----------------------------\n"
    f"addition (a + b): {MathCS207.add(**inputs)}\n"
    f"subtraction (a - b): {MathCS207.subtract(**inputs)}\n"
    f"multiplication (a*b): {MathCS207.multiply(**inputs)}\n"
    f"division (a/b): {MathCS207.divide(**inputs)}"
)

inputs: {'a': 1.0, 'b': 2.0}
----------------------------
addition (a + b): 3.0
subtraction (a - b): -1.0
multiplication (a*b): 2.0
division (a/b): 0.5


### Part 3: Import a single function from a module and use it

In [49]:
from MathCS207 import add

print(add(**inputs))

3.0


### Part 4: Import a module by creating an alias of it and then use the alias

In [50]:
import MathCS207 as mathcs

print(mathcs.add(**inputs))

3.0


### Part 5: List every function definition inside the module MathCS207

In [52]:
print(dir(MathCS207))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'add', 'divide', 'multiply', 'subtract']


## Problem 3: Bank Account Revisited [50pts]

###### Defining `SAVINGS` and `CHECKING` bank accounts

In [67]:
from enum import Enum
class AccountType(Enum):
    SAVINGS = 1
    CHECKING = 2

### Part 1: Create a BankAccount class

In [ ]:
class BankAccount():
    
    def __init__(self, owner, accountType):
        self.owner = owner
        self.accountType = accountType
        
    def withdraw(self, amount): 
        return 
    
    def deposit(self, amount):

### Part 2: Write a class BankUser

In [ ]:
class BankUser():
    
    def __init__(self, owner):
        self.owner = owner
    
    def addAccount(self, accountType):
        
    def getBalance(self, accountType):
    
    def deposit(self, accountType, amount): 
        
    def withdraw(self, accountType, amount):

In [76]:
def make_withdraw(balance):
    def update_balance(debit):
        # places the variable 'balance' in the outer scope of
        # make_withdraw 
        # into the inner scope of update_balance
        nonlocal balance
        if debit <= balance:
            balance -= debit
            print(f"New balance: ${balance:.2f}")
        else:
            print("Please enter a withdrawl amount " 
                  "less than or equal to your current balance.")
    return update_balance

init_balance = 500 # initial balance
withdraw_amount = 150.50 # amount to withdraw

wd = make_withdraw(init_balance)
wd(withdraw_amount)

New balance: $349.50
